In [2]:
from icecube import dataclasses, icetray, dataio, clsim, dst
from icecube import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import os
from datetime import datetime, timezone
import numpy as np
import json

### select day and device

In [3]:
day = '0112'
devices = ['str87_dev8','str88_dev6','str88_dev78']
device = devices[0]

### load files and time windows of run

In [4]:
det_direct = '/data/ana/Calibration/POCAM/data/freezein/detector/'
file_path = os.path.join(det_direct,day)
files = [file for file in os.listdir(file_path) if file.endswith(".i3")]
files.sort()
print(files)


['PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000000.i3', 'PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000001.i3', 'PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000002.i3', 'PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000003.i3', 'PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000004.i3', 'PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000005.i3', 'PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000006.i3', 'PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000007.i3', 'PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000008.i3']


In [5]:
dev_dir = '/data/ana/Calibration/POCAM/data/freezein/device/'
dev_path = os.path.join(dev_dir,day)
time_file = 'run_start_end_emitter_start_end_'+device+'.json'
with open(os.path.join(dev_path,time_file), 'r') as file:
    time_stamps = json.load(file)

In [23]:
run_start = datetime.strptime(time_stamps['run_start'], "%Y-%m-%d %H:%M:%S,%f").replace(tzinfo=timezone.utc)
run_end =  datetime.strptime(time_stamps['run_end'], "%Y-%m-%d %H:%M:%S,%f").replace(tzinfo=timezone.utc)

print(run_start)
print(run_end)

2026-01-12 19:23:45.290000+00:00
2026-01-12 19:25:27.373000+00:00


### load data

In [7]:
neighbors = {'87':[79,36,85],'88':[79,36,80],'89':[79,85],'90':[79,80,84],'91':[80,83],'92':[80,36],'93':[79,36]}

### search for needed files for specific device run

In [16]:
for file in files:
    print(f'filename = {file}')
    i3_file = dataio.I3File(os.path.join(file_path,file))
    frame = i3_file.pop_frame()
    event = dataclasses.I3EventHeader.from_frame(frame,"I3EventHeader")
    print(event.start_time.date_time)
    #print(event.end_time.date_time)
    print('\n')

filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000000.i3
2026-01-12 19:21:55.540407+00:00


filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000001.i3
2026-01-12 19:23:00.937647+00:00


filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000002.i3
2026-01-12 19:24:05.711338+00:00


filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000003.i3
2026-01-12 19:25:05.934085+00:00


filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000004.i3
2026-01-12 19:26:10.441637+00:00


filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000005.i3
2026-01-12 19:27:12.282427+00:00


filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000006.i3
2026-01-12 19:28:16.472789+00:00


filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000007.i3
2026-01-12 19:28:54.674728+00:00


filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000008.i3
2026-01-12 19:29:41.612966+00:00




### extract data from all needed files

In [24]:
def get_omkey(s_id,d_id):
    omkey = pulses.get(icetray.OMKey(s_id,d_id))
    return omkey
def get_z(s_id,d_id):
    pos = -pulses.get(icetray.OMKey(s_id,d_id)).position[2]#+1948.07
    return pos

#charges_per_module_time = pd.DataFrame()
devstr = device.split('_')[0][-2:]
strings = neighbors[devstr]
timecharges_per_module={}

for string in strings:
    timecharges_per_module[str(string)] = {}
    for d in np.arange(1,61):
        timecharges_per_module[str(string)][str(d)] = {}

for file in files[1:4]:
    print(f'filename = {file}')
    i3_file = dataio.I3File(os.path.join(file_path,file))
    
    while i3_file.more():
        frame = i3_file.pop_frame()
        pulses = dataclasses.I3RecoPulseSeriesMap.from_frame(frame, "I3SuperDST")
        event = dataclasses.I3EventHeader.from_frame(frame,"I3EventHeader")
        event_time = event.start_time.date_time
        dt_ns = int((event_time - run_start).total_seconds() * 1e9)
        end_check = int((event_time - run_end).total_seconds() * 1e9)
        if not (dt_ns > 0 and end_check < 0) :
            continue
        for string in strings:
            for i, m in enumerate(np.arange(1,61)):
                if get_omkey(string,int(m)) == None:
                    pass
                else:
                    reco_pulse = get_omkey(string,int(m))
                    for pulse in reco_pulse:
                        timecharges_per_module[str(string)][str(m)][str(dt_ns + pulse.time)] = pulse.charge


filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000001.i3
filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000002.i3
filename = PFFilt_PhysicsFiltering_Run00141896_Subrun00000000_00000003.i3


### select saving path

In [25]:
save_path = os.path.join('/data/user/leidensc/upgrade_installation_ops/freeze_in/',day)

### save data    only needed if new data needs to be converted

In [26]:
with open(os.path.join(save_path,device), 'w') as file:
    json.dump(timecharges_per_module, file, indent=4)